In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import torch

import utils
import identification_model


plt.style.use("dark_background")

In [ ]:
use_preprocessed_data = True
batch_size = 64

if use_preprocessed_data:
    model_dir, batch_iter = utils.init_identification_training(batch_size, use_preprocessed_data=True)
    batch_inputs, batch_labels = batch_iter.next_batch()
    batch_inputs = ((batch_inputs.type(torch.FloatTensor) / 255) - torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)) / torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
    plt.figure(figsize=(8,8))
    plt.imshow((np.transpose(batch_inputs[0].numpy(), (1, 2, 0))*np.array([0.229, 0.224, 0.225]))+np.array([0.485, 0.456, 0.406]))

else:
    model_dir, data_dirs, dataloaders, class_names = utils.init_identification_training(batch_size, use_preprocessed_data=False)
    batch_inputs, batch_classes = next(iter(dataloaders[data_dirs["train"]]))
    plt.figure(figsize=(8,8))
    plt.imshow((np.transpose(batch_inputs[0].numpy(), (1, 2, 0))*np.array([0.229, 0.224, 0.225]))+np.array([0.485, 0.456, 0.406]))

In [ ]:
nb_of_classes = 415

network, criterion, optimizer = identification_model.init_model(model_dir, nb_of_classes)

In [ ]:
unknown_id = 414
num_epochs = 80

if use_preprocessed_data:
    network, best_acc, train_losses, val_losses = identification_model.train_batch_iterator(model_dir, network, criterion, optimizer, batch_iter, unknown_id, num_epochs)
else:
    network, best_acc, train_losses, val_losses = identification_model.train(model_dir, network, criterion, optimizer, data_dirs, dataloaders, unknown_id, num_epochs)

In [ ]:
utils.save_identification_model(model_dir, network, best_acc, train_losses, val_losses)

In [ ]:
if use_preprocessed_data:
    utils.evaluate_identification_batch_iter(model_dir, network)
else:
    utils.evaluate_identification(model_dir, network, data_dirs, dataloaders, class_names)